In [ ]:
#
#
#

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
data = pd.read_csv('data/reloading_data - 9mm parabellum.csv')
data.head()

In [ ]:
data.shape

In [ ]:
# add a "bullet type" column

orig_columns =                           ['powder', 'start', 'velocity', 'never exceed', 'velocity max', 'press', 'units', 'min oal']
new_columns  = ['bul_weight', 'bul_type', 'powder', 's_load', 's_velocity', 'ne_load', 'ne_velocity', 'press', 'units', 'oal']

lee = pd.DataFrame(columns = new_columns)

bul_weight = None
bul_type = None
for pos in range(0, data.shape[0]):
    line = tuple(data.loc[pos, orig_columns].array)
    # print(123, line) # DEBUG
    if ' grain ' in line[0]: # this is a bullettype "115 grain jacketed"
        bul_weight, bul_type = line[0].split(' grain ')
    else:
        line = (bul_weight, bul_type, *line)
        lee.loc[len(lee.index)] = line


In [ ]:
lee

In [ ]:
lee.shape

In [ ]:
powder_labels = lee.powder.unique()
print(powder_labels)
bul_types = lee.bul_type.unique()
print(bul_types)

In [ ]:
def powder_id(x):
    return np.where(lee.powder.unique() == x)[0][0]

def bul_type_id(x):
    return np.where(lee.bul_type.unique() == x)[0][0]

In [ ]:
lee['powder_id'] = [ powder_id(x) for x in lee.powder ]
lee['bul_type_id'] = [ bul_type_id(x) for x in lee.bul_type ]

In [ ]:
lee

In [ ]:
lee[lee.powder == 'HP38']

In [ ]:
lee_clean = lee.dropna()

In [ ]:
msk = np.random.rand(len(lee_clean)) < 0.8
train = lee_clean[msk]
test = lee_clean[~msk]

train.describe()

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model

y_keys = ['s_load', 'ne_load']
x_keys = ['powder_id', 'bul_type_id' ]

train_x = np.asanyarray(train[x_keys])
train_y = np.asanyarray(train[y_keys])

test_x = np.asanyarray(test[x_keys])
test_y = np.asanyarray(test[y_keys])
print(train_x.shape)

poly = PolynomialFeatures(degree=5)
train_x_poly = poly.fit_transform(train_x)
train_x_poly.shape

In [ ]:
clf = linear_model.LinearRegression()
train_y_ = clf.fit(train_x_poly, train_y)
# The coefficients
print ('Coefficients: ', clf.coef_)
print ('Intercept: ',clf.intercept_)

In [ ]:
from sklearn.metrics import r2_score

test_x_poly = poly.fit_transform(test_x)
test_y_ = clf.predict(test_x_poly)

print("Mean absolute error: %.2f" % np.mean(np.absolute(test_y_ - test_y)))
print("Residual sum of squares (MSE): %.2f" % np.mean((test_y_ - test_y) ** 2))
print("R2-score: %.2f" % r2_score(test_y_ , test_y) )

In [ ]:
test_y_

In [ ]:
# Let's do actual calculations

actual_x = np.array([[23, 0], [23, 1], [23, 2], [23, 3]])
actual_x

In [ ]:
actual_x_poly = poly.fit_transform(actual_x)
actual_y_ = clf.predict(actual_x_poly)

In [ ]:
actual_y_

In [ ]:
lee.bul_type.unique()